# 01 — Data Collection & Panel Dataset

**Goal:** Pull price data, earnings history, and macro regime data for the full S&P 500. 
Merge everything into one flat panel dataset where **each row = one earnings event**.

**Data Sources:**
- `yfinance` — Price/Volume, ^TNX (10Y Yield), ^VIX
- `yfinance` earnings history — EPS Actual vs Estimate (proxy for Capital IQ)
- *(If you have Capital IQ access, swap in true consensus dispersion for SUE)*

**Output:** `data/panel_dataset.csv`

In [ ]:
import sys
sys.path.append('..')  # so we can import from utils/

import pandas as pd
import numpy as np
import yfinance as yf
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

from utils.helpers import (
    get_sp500_tickers, get_price_data, get_spx_returns,
    get_macro_data, get_earnings_history,
    compute_technical_features, compute_ticker_history_features,
    compute_targets, compute_magnitude_class
)

import os
os.makedirs('../data', exist_ok=True)
os.makedirs('../outputs', exist_ok=True)

In [ ]:
# ── Configuration ──────────────────────────────────────────────
START_DATE     = '2019-01-01'
END_DATE       = '2024-12-31'
POST_DAYS      = 5       # hold period for return calculation
PRE_WINDOW     = 30      # trading days of pre-earnings price history
MIN_PRICE      = 5.0     # skip penny stocks
MAX_TICKERS    = 100     # set to None for full S&P 500 (takes ~45 min)

print(f'Date range : {START_DATE} → {END_DATE}')
print(f'Post-earnings hold : {POST_DAYS} days')
print(f'Tickers    : {"All S&P 500" if MAX_TICKERS is None else MAX_TICKERS} (for testing)')

In [ ]:
# ── Step 1: Load macro data (rates + VIX) ─────────────────────
print('Downloading macro data (^TNX, ^VIX)...')
macro = get_macro_data(start=START_DATE, end=END_DATE)
print(f'Macro data shape: {macro.shape}')
macro.head()

In [ ]:
# ── Step 2: Load S&P 500 returns for alpha calculation ─────────
print('Downloading S&P 500 index returns...')
spx = get_spx_returns(start=START_DATE, end=END_DATE)
print(f'SPX data shape: {spx.shape}')

In [ ]:
# ── Step 3: Get ticker universe ────────────────────────────────
print('Fetching S&P 500 tickers...')
all_tickers, sector_map = get_sp500_tickers()
tickers = all_tickers[:MAX_TICKERS] if MAX_TICKERS else all_tickers
print(f'Tickers loaded: {len(tickers)}')

In [ ]:
# ── Step 4: Main loop — build one row per earnings event ───────
records = []
skipped = []

for ticker in tqdm(tickers, desc='Processing tickers'):
    
    # --- Price data ---
    price = get_price_data(ticker, start=START_DATE, end=END_DATE)
    if price is None or price['close'].iloc[-1] < MIN_PRICE:
        skipped.append((ticker, 'no price data'))
        continue

    # --- Earnings history ---
    earn = get_earnings_history(ticker)
    if earn is None or len(earn) < 4:
        skipped.append((ticker, 'insufficient earnings history'))
        continue

    # Filter to our date range
    earn = earn[(earn.index >= START_DATE) & (earn.index <= END_DATE)]
    if len(earn) < 2:
        skipped.append((ticker, 'no earnings in date range'))
        continue

    # --- Per-event feature extraction ---
    all_earn_dates = earn.index.tolist()

    for earn_date in all_earn_dates:
        try:
            # Pre-earnings price window
            pre = price[price.index < earn_date].tail(PRE_WINDOW)
            if len(pre) < 20:
                continue
            prev_close = pre.iloc[-1]['close']

            # Post-earnings return (stock)
            post = price[price.index > earn_date]
            if len(post) < POST_DAYS:
                continue
            post_close   = post.iloc[POST_DAYS - 1]['close']
            stock_ret_5d = post_close / prev_close - 1

            # Post-earnings return (S&P 500) — same window
            spx_window = spx[
                (spx.index > earn_date)
            ].head(POST_DAYS)
            if len(spx_window) < POST_DAYS:
                continue
            spx_ret_5d = (
                spx_window.iloc[-1]['spx_close'] /
                spx_window.iloc[0]['spx_close'] - 1
            )

            # Target variables
            target_binary, excess_ret = compute_targets(stock_ret_5d, spx_ret_5d)
            target_mag = compute_magnitude_class(excess_ret)

            # Macro regime on earnings date
            macro_on_date = macro[macro.index <= earn_date]
            if macro_on_date.empty:
                continue
            macro_row = macro_on_date.iloc[-1]

            # Technical features
            tech = compute_technical_features(pre)

            # Ticker historical earnings behavior
            past_dates = [d for d in all_earn_dates if d < earn_date]
            hist_feats = compute_ticker_history_features(price, past_dates)

            # Earnings surprise row
            earn_row = earn.loc[earn_date]

            # Assemble record
            row = {
                # Identifiers
                'ticker':         ticker,
                'earn_date':      earn_date,
                'sector':         sector_map.get(ticker, 'Unknown'),
                'quarter':        earn_date.quarter,
                'year':           earn_date.year,

                # Targets
                'target_binary':  target_binary,
                'target_mag':     target_mag,
                'stock_ret_5d':   stock_ret_5d,
                'spx_ret_5d':     spx_ret_5d,
                'excess_ret_5d':  excess_ret,

                # Earnings surprise signals
                'sue':              earn_row.get('sue', np.nan),
                'eps_surprise_pct': earn_row.get('eps_surprise_pct', np.nan),
                'hist_beat_rate':   earn_row.get('hist_beat_rate', np.nan),
                'beat_streak':      earn_row.get('beat_streak', np.nan),

                # Macro regime
                'tnx':              macro_row['tnx'],
                'vix':              macro_row['vix'],
                'rate_regime':      macro_row['rate_regime'],
                'vix_regime':       macro_row['vix_regime'],
                'tnx_1m_chg':       macro_row['tnx_1m_chg'],
                'vix_5d_chg':       macro_row['vix_5d_chg'],
            }

            # Add technical features
            row.update(tech)
            # Add historical behavior features
            row.update(hist_feats)

            records.append(row)

        except Exception as e:
            continue  # skip malformed events silently

print(f'\nTotal events collected : {len(records)}')
print(f'Tickers skipped        : {len(skipped)}')

In [ ]:
# ── Step 5: Build DataFrame and save ──────────────────────────
panel = pd.DataFrame(records)
panel = panel.sort_values('earn_date').reset_index(drop=True)

print(f'Panel shape: {panel.shape}')
print(f'Date range : {panel["earn_date"].min()} → {panel["earn_date"].max()}')
print(f'Unique tickers: {panel["ticker"].nunique()}')
print(f'\nClass balance (target_binary):')
print(panel['target_binary'].value_counts(normalize=True).round(3))

panel.to_csv('../data/panel_dataset.csv', index=False)
print('\n✓ Saved to data/panel_dataset.csv')

In [ ]:
# ── Step 6: Data quality check ────────────────────────────────
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
fig.suptitle('Panel Dataset — Data Quality Overview', fontsize=14, y=1.02)

# Events over time
panel['earn_date'] = pd.to_datetime(panel['earn_date'])
panel.groupby(panel['earn_date'].dt.to_period('Q')).size().plot(
    ax=axes[0,0], title='Events per Quarter', kind='bar', color='steelblue'
)
axes[0,0].tick_params(axis='x', rotation=45)

# Target balance
panel['target_binary'].value_counts().plot(
    ax=axes[0,1], title='Target Balance (0=Under, 1=Over)', kind='bar',
    color=['tomato','steelblue']
)

# Excess return distribution
axes[0,2].hist(panel['excess_ret_5d'].clip(-0.3, 0.3), bins=60, color='steelblue', edgecolor='white')
axes[0,2].axvline(0, color='red', linestyle='--')
axes[0,2].set_title('Excess Return Distribution (5d)')

# Events per sector
panel['sector'].value_counts().plot(
    ax=axes[1,0], title='Events by Sector', kind='barh', color='steelblue'
)

# Rate regime over time
rate_by_q = panel.groupby(panel['earn_date'].dt.to_period('Q'))['rate_regime'].mean()
rate_by_q.plot(ax=axes[1,1], title='Rate Regime Over Time (1=High Rate)', color='darkorange')
axes[1,1].axhline(0.5, color='gray', linestyle='--', alpha=0.5)

# Missing value heatmap
missing = panel.isnull().mean().sort_values(ascending=False).head(15)
missing.plot(ax=axes[1,2], title='Missing Value Rate (top 15 cols)', kind='barh', color='tomato')

plt.tight_layout()
plt.savefig('../outputs/01_data_quality.png', dpi=150, bbox_inches='tight')
plt.show()
print('Plot saved to outputs/01_data_quality.png')